## Implement CBC mode

CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

[The file here](https://www.cryptopals.com/static/challenge-data/10.txt) is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

### Don't cheat.
Do not use OpenSSL's CBC code to do CBC mode, even to verify your results. What's the point of even doing this stuff if you aren't going to learn from it?


## Test AES primitive
Make sure PyCryptodome API can be used for a hand-rolled CBC mode.

In [1]:
from Crypto.Cipher import AES

In [2]:
key = b"YELLOW SUBMARINE"

In [3]:
cipher = AES.new(key, AES.MODE_ECB)

Single block of plaintext.

In [4]:
test_data = b"some secret text"
len(test_data)

16

In [5]:
ciphertext = cipher.encrypt(test_data)
print(ciphertext.hex(), len(ciphertext))

c4e42ecdc5232819db7d5fb468e56ac9 16


In [6]:
assert test_data == cipher.decrypt(ciphertext)

### Test CBC mode interaction with first plaintext block

In [7]:
from os import urandom
from pwn import xor

In [8]:
iv = urandom(cipher.block_size)
iv.hex()

'692051e0b261426d673a3ee101623bf7'

In [9]:
xor(iv, test_data)

b"\x1aO<\x85\x92\x12'\x0e\x15_J\xc1u\x07C\x83"

$C_i = E(K, P_i \oplus C_{i-1})$

In [10]:
c = cipher.encrypt( xor(test_data, iv) )
c.hex()

'f43f2bd87b701d021a704e419d534de5'

$P_i = D(K, C_i) \oplus C_{i-1}$

In [11]:
xor(cipher.decrypt(c), iv)

b'some secret text'

### Parse challenge file

In [13]:
data = None
with open('10.txt', 'r') as f:
    data = f.read()

In [14]:
data = data.replace('\n', '')
data

'CRIwqt4+szDbqkNY+I0qbNXPg1XLaCM5etQ5Bt9DRFV/xIN2k8Go7jtArLIyP605b071DL8C+FPYSHOXPkMMMFPAKm+Nsu0nCBMQVt9mlluHbVE/yl6VaBCjNuOGvHZ9WYvt51uR/lklZZ0ObqD5UaC1rupZwCEK4pIWf6JQ4pTyPjyiPtKXg54FNQvbVIHeotUG2kHEvHGS/w2Tt4E42xEwVfi29J3yp0O/TcL7aoRZIcJjMV4qxY/uvZLGsjo1/IyhtQp3vY0nSzJjGgaLYXpvRn8TaAcEtH3cqZenBooxBH3MxNjD/TVf3NastEWGnqeGp+0D9bQx/3L0+xTf+k2VjBDrV9HPXNELRgPN0MlNo79p2gEwWjfTbx2KbF6htgsbGgCMZ6/iCshy3R8/abxkl8eK/VfCGfA6bQQkqs91bgsT0RgxXSWzjjvh4eXTSl8xYoMDCGa2opN/b6Q2MdfvW7rEvp5mwJOfQFDtkv4M5cFEO3sjmU9MReRnCpvalG3ark0XC589rm+42jC4/oFWUdwvkzGkSeoabAJdEJCifhvtGosYgvQDARUoNTQAO1+CbnwdKnA/WbQ59S9MU61QKcYSuk+jK5nAMDot2dPmvxZIeqbB6ax1IH0cdVx7qB/Z2FlJ/U927xGmC/RUFwoXQDRqL05L22wEiF85HKx2XRVB0F7keglwX/kl4gga5rk3YrZ7VbInPpxUzgEaE4+BDoEqbv/rYMuaeOuBIkVchmzXwlpPORwbN0/RUL89xwOJKCQQZM8B1YsYOqeL3HGxKfpFo7kmArXSRKRHToXuBgDq07KS/jxaS1a1Paz/tvYHjLxwY0Ot3kS+cnBeq/FGSNL/fFV3J2a8eVvydsKat3XZS3WKcNNjY2ZEY1rHgcGL5bhVHs67bxb/IGQleyY+EwLuv5eUwS3wljJkGcWeFhlqxNXQ6NDTzRNlBS0W4CkNiDBMegCcOlPKC2ZLGw2ejgr2utoNfmRtehr

In [15]:
from base64 import b64decode

In [17]:
ciphertext = b64decode(data)
len(ciphertext)

2880

Length is a multiple of block size so likely properly padded.

In [18]:
assert len(ciphertext) % cipher.block_size == 0

Challenge specifies an IV of all ASCII zeros.

In [20]:
iv = b'\x00' * cipher.block_size
iv

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

## DIY CBC mode

In [23]:
plaintext = b''
for i in range(0, len(ciphertext), cipher.block_size):
    block = ciphertext[i:i+cipher.block_size]
    previous_block = iv
    if i != 0:
        previous_block = ciphertext[i-cipher.block_size:i]
    
    plaintext += xor(cipher.decrypt(block), previous_block)
    

In [24]:
plaintext

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

In [25]:
plaintext[-cipher.block_size:]

b'unky music \n\x04\x04\x04\x04'